# Modelos de predicción


In [1]:
# importando librerias

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import ensemble

In [2]:
df = pd.read_csv('datasets/BlackFriday.csv', sep=',')

In [3]:
# eliminar columnas innecesarias

df.drop('Product_Category_2', axis=1, inplace=True)
df.drop('Product_Category_3', axis=1, inplace=True)

In [4]:
from sklearn.model_selection import train_test_split

# dividir dataset para entrenamiento y pruebas
df_train, df_test = train_test_split(df)

In [5]:
df_train.to_csv("datasets/train.csv", index=False)
df_test.to_csv("datasets/test.csv", index=False)

In [6]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,7969


In [7]:
print(df.shape)
print(df_train.shape)
print(df_test.shape)

(537577, 10)
(403182, 10)
(134395, 10)


In [8]:
# columnas categorias
categorical_columns = ["Product_ID", "Gender", "Age", "City_Category", "Stay_In_Current_City_Years"]


In [9]:
# seleccionando el target y separando los datos

train_y = np.array(df_train["Purchase"])
test_y = np.array(df_test["Purchase"])

train_X = df_train.copy()
test_X = df_test.copy()

train_X.drop('Purchase', axis=1, inplace=True)
test_X.drop('Purchase', axis=1, inplace=True)

In [10]:
train_X.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
515388,1001377,P00267542,M,36-45,16,B,3,1,1
338544,1004079,P00085142,M,36-45,17,B,3,1,5
325656,1002090,P00196142,M,36-45,17,C,2,0,3
416820,1004107,P00059442,F,26-35,0,B,1,0,6
141036,1003769,P00193842,M,26-35,15,B,0,0,11


In [11]:
test_X.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
452483,1003688,P00251242,M,18-25,6,B,1,0,5
90779,1001948,P00229742,M,0-17,10,B,3,0,1
508852,1000372,P00214442,F,18-25,4,C,2,0,8
72734,1005175,P00249342,F,36-45,9,C,1,0,5
329088,1002737,P00194542,M,26-35,3,B,3,0,8


In [12]:
type(df["Product_ID"][0])

str

In [13]:
# Normalizando los datos
# en lugar de usar la funcion map como vimos antes, vamos a encodificar las variables categoricas

for var in categorical_columns:
    lb = LabelEncoder()
    full_var_data = pd.concat((train_X[var], test_X[var]), axis=0).astype('str')
    lb.fit( full_var_data )
    train_X[var] = lb.transform(train_X[var].astype('str'))
    test_X[var] = lb.transform(test_X[var].astype('str'))

In [14]:
train_X.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
515388,1001377,2557,1,3,16,1,3,1,1
338544,1004079,823,1,3,17,1,3,1,5
325656,1002090,1855,1,3,17,2,2,0,3
416820,1004107,581,0,2,0,1,1,0,6
141036,1003769,1832,1,2,15,1,0,0,11


In [15]:
train_y

array([11710,  8846,  8193, ..., 20793, 11784,  5409])

In [16]:
print("Train shape is : ", train_X.shape)
print("Test shape is : ", test_X.shape)

Train shape is :  (403182, 9)
Test shape is :  (134395, 9)


In [17]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.05
params["seed"] = 0
params["max_depth"] = 3
plst = list(params.items())

In [18]:
xgtrain = xgb.DMatrix(train_X, label=train_y)
xgtest = xgb.DMatrix(test_X)


In [19]:
num_rounds = 100 # 5667
model = xgb.train(plst, xgtrain, num_rounds)
pred_test_y_xgb1 = model.predict(xgtest)


# submission
# test['Purchase']=pred_test_y_xgb1
# test.to_csv('Solution.csv',columns = ['User_ID','Product_ID','Purchase'],index = False)

columns_out = ['User_ID','Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 
               'Stay_In_Current_City_Years','Marital_Status', 'Product_Category_1','Purchase']

test_X['Purchase']=pred_test_y_xgb1
test_X.to_csv('datasets/Solution.csv',columns = columns_out, index = False)

[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:20] /work

[10:57:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:57:24] /work

Ya con nuestros datos de predicción podemos volver a realizar las mismas graficas que en el notebook 2.Analisis_previo y hacer una modificación del precio proporsional con el valor de la TRM predicha.

In [20]:
# guardar codificacion de datos de entrenamiento
train_X['Purchase']=train_y
train_X.to_csv('datasets/train_encode.csv')